# Identify toxicity comments

# 1. Description
- A main area of focus is machine learning models that can identify toxicity in online conversations, where toxicity is defined as anything rude, disrespectful or otherwise likely to make someone leave a discussion. If these toxic contributions can be identified, we could have a safer, more collaborative internet.

- `Disclaimer`: The dataset for this competition contains text that may be considered profane, vulgar, or offensive.

# 2. Evaluation
- `area under the ROC curve` between the predicted probability and the observed target.

# 3. Data
- `comment_text`: This contains the text of a comment which has been classified as toxic or non-toxic (0...1 in the toxic column). The data set’s comments are entirely in english and come either from Civil Comments or Wikipedia talk page edits.

- What am I predicting?
> You are predicting the probability that a comment is toxic. A toxic comment would receive a 1.0. A benign, non-toxic comment would receive a 0.0. In the test set, all comments are classified as either a 1.0 or a 0.0.

In [ ]:
!pip install -q pyicu
!pip install -q pycld2
!pip install -q polyglot
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import tqdm
import transformers
import tensorflow as tf

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split

from polyglot.detect import Detector

import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)
print(transformers.__version__)
print(tf.keras.__version__)

In [ ]:
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Data Preprocessing

In [ ]:
def get_language(text):
    return Detector(
        "".join(x for x in text if x.isprintable()), quiet=True
    ).languages[0].name

PATH =  "../input/jigsaw-multilingual-toxic-comment-classification"
FILES = os.listdir(PATH)
print(FILES)

TRAIN_PATH = os.path.join(PATH, 'jigsaw-toxic-comment-train.csv')
data = pd.read_csv(TRAIN_PATH)

data["lang"] = data["comment_text"].apply(get_language)
data = data[data['lang'] == 'English']

In [ ]:
data.head()

In [ ]:
data.toxic.value_counts()

# Split data to Train and Test

In [ ]:
X = data[['comment_text']]
y = data[['toxic']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_test = y_test.toxic.values
y_train = y_train.toxic.values

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Tokenizing the data

In [ ]:
def map_func(input_ids, masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': masks
    }, labels

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = transformers.AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

SEQ_LEN = 128
X_train_ids = np.zeros((len(X_train), SEQ_LEN))
X_train_mask = np.zeros((len(X_train), SEQ_LEN))

X_test_ids = np.zeros((len(X_test), SEQ_LEN))
X_test_mask = np.zeros((len(X_test), SEQ_LEN))

for i, sequence in enumerate(X_train['comment_text']):
    tokens = tokenizer.encode_plus(
        sequence, max_length=SEQ_LEN,
        truncation=True, padding='max_length',
        add_special_tokens=True, return_token_type_ids=False,
        return_attention_mask=True, return_tensors='tf'
    )
    X_train_ids[i, :], X_train_mask[i, :] = tokens['input_ids'], tokens['attention_mask']
    
for i, sequence in enumerate(X_test['comment_text']):
    tokens = tokenizer.encode_plus(
        sequence, max_length=SEQ_LEN,
        truncation=True, padding='max_length',
        add_special_tokens=True, return_token_type_ids=False,
        return_attention_mask=True, return_tensors='tf'
    )
    X_test_ids[i, :], X_test_mask[i, :] = tokens['input_ids'], tokens['attention_mask']
    
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_ids, X_train_mask, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_ids, X_test_mask, y_test))

train_dataset = train_dataset.map(map_func)
test_dataset = test_dataset.map(map_func)

train_dataset = train_dataset.shuffle(100000).batch(32, drop_remainder=True)
test_dataset = test_dataset.shuffle(100000).batch(32, drop_remainder=True)

# Model Building

In [ ]:
bert = transformers.TFAutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN, ), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN, ), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]

X = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
# X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
# X = tf.keras.layers.BatchNormalization()(X)
# X = tf.keras.layers.Dense(128, activation='relu')(X)
# X = tf.keras.layers.Dropout(0.1)(X)
# X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6), 
    loss='binary_crossentropy', 
    metrics=[tf.keras.metrics.AUC(name='AUC')]
)

r = model.fit(
    train_dataset,
    validation_data=(test_dataset),
    epochs=10,
    batch_size=4096
)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
def plot_learning_evolution(r):
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.plot(r.history['loss'], label='Loss')
    plt.plot(r.history['val_loss'], label='val_Loss')
    plt.title('Loss evolution during trainig')
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.plot(r.history['AUC'], label='AUC')
    plt.plot(r.history['val_AUC'], label='val_AUC')
    plt.title('AUC score evolution during trainig')
    plt.legend();

In [ ]:
plot_learning_evolution(r)

In [ ]:
# y_pred = model.predict(test_dataset)
# confusion_matrix(y_test, y_pred.round())

# Saving and Uploading the model

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
tf.keras.models.save_model(model, "hate_speech_10_epochs.hdf5")
model.save("hate_speech_10_epochs.h5")

In [ ]:
hdf5_model = tf.keras.models.load_model("hate_speech_10_epochs.hdf5")
hdf5_model.summary()

In [ ]:
h5_model = tf.keras.models.load_model("hate_speech_10_epochs.h5")
h5_model.summary()

In [ ]:
def prep_sentence(sentence):
    tokens = tokenizer.encode_plus(
        sentence, max_length=SEQ_LEN,
        truncation=True, padding='max_length',
        add_special_tokens=True, return_token_type_ids=False,
        return_attention_mask=True, return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(tokens['input_ids'], tf.float64),
        'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)
    }

# Toxic Comments

In [ ]:
toxic_speechs = [
    'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK',
    'MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MOVIES. HE HAS SO MUCH BUTTSEX THAT HIS ASSHOLE IS NOW BIG ENOUGH TO BE CONSIDERED A COUNTRY.',
    "A block ohhhhhhhhhhhhhh noooooooooooo I'm soooo like gonna cry and like shit ... ha ha.  you think i care?  i dont even use wikipedia.  look at the serb reporting me to the geek squad, what are you like 5?  Rumor has it that you are another canadian serb.  Rumor has it that you have pissed of a select few from B93 & WP.   )  BYE BYE.",
    "it is a constructive edit you idiot, every kid of every age should know that santa claus is fucking fictional. ever since i first heard of santa claus i knew that he was fictional, my parents didn't give me any delusions and if they had, i would've laughed in their faces and said it isn't logical because it fucking isn't. every kid should be logical just like i was and every kid should be able to logically fucking infer that there is no fucking santa claus in the real human universe.",
    'honestly ==\nyou need to crawl under a rock and DIE YOU FAT BASTARD\n\n=='
]

In [ ]:
for speech in toxic_speechs:
    prediction = h5_model.predict(prep_sentence(speech))
    print(prediction)

In [ ]:
for speech in toxic_speechs:
    prediction = hdf5_model.predict(prep_sentence(speech))
    print(prediction)

# Non-Toxic Comments

In [ ]:
non_toxic_speechs = [
    "Gale, you're living proof why wikipedia should NEVER be trusted as fact. I mean, telling someone to blindly believe whatever's said instead of verifying? You need to take a walk in traffic for saying that!\n\n99.149.119.168",
    'EastEnders Manual of Style \n\nHello, just wanted you to be aware of the EE MoS, which helps us work out what is appropriate for Infoboxes etc.  Cheers,  (Talk)',
    'You need to provide high-quality secondary sources (e.g., not original publications from medical experiments, but perhaps review articles or medical textbooks) that support this significant change in definition.',
    "I appreciate your responses, guys. I take the recommendation as an admin as a great compliment. However, since I move around so much and my knowledge of Wikipedia isn't where I would like it to be before I went after something like that, I will get back to you if the vote ever happens. In the mean time, I definitely appreciate the compliment. }",    
    "Stop reinserting harrassing content on WP:ANI \n\nStop readding this material.  If you continue with this from other IP ranges or addresses we will be forced to block larger IP ranges from editing.  You aren't allowed to harrass people like this on Wikipedia."
]

In [ ]:
for speech in non_toxic_speechs:
    prediction = h5_model.predict(prep_sentence(speech))
    print(prediction)

In [ ]:
for speech in non_toxic_speechs:
    prediction = hdf5_model.predict(prep_sentence(speech))
    print(prediction)